Импорт библиотек

In [112]:
import requests as r
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pt

Задание 3

In [113]:
#Подавление ошибки цепного назначения
pd.options.mode.chained_assignment = None

#Раскрываем все строки и столбцы
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#Запрашиваем данные через GET запрос и преобразуем в json
visits_json = r.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01').json()
registrations_json = r.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01').json()

#Преобразовываем json в DataFrame
visits = pd.DataFrame(visits_json)
registrations = pd.DataFrame(registrations_json)

#Добавляем столбец с датой
visits['date'] = pd.to_datetime(visits['datetime']).dt.date
registrations['date'] = pd.to_datetime(registrations['datetime']).dt.date

#Выбираем дату последнего визита каждого пользователя
visits["datetime"] = pd.to_datetime(visits["datetime"], format="%a, %d %b %Y %H:%M:%S %Z")
visits["last_datetime"] = visits.groupby('visit_id')["datetime"].transform('max')
last_visits = visits[visits['datetime'] == visits['last_datetime']]

#Маркируем ботов
# def is_bot(row):
#     return 1 if row['user_agent'].str.contains('bot') else 0

# last_visits['is_bot'] = last_visits['user_agent'].apply(is_bot, axis = 1)

last_visits['is_bot'] = np.where(last_visits['user_agent'].str.contains('bot'), 'yes', 'no')

#Группируем и объединяем данные
visits_and_registrations = pd.concat([
    last_visits[last_visits['is_bot'] == 'no'].groupby(['date', 'platform']).agg({'visit_id': 'count'}),
    registrations.groupby(['date', 'platform']).agg({'user_id': 'count'})
], axis=1).reset_index()

#Переименовываем столбцы и заполняем нули
visits_and_registrations.rename(columns = {'visit_id':'visits', 'user_id':'registrations'}, inplace = True )
visits_and_registrations['registrations'] = visits_and_registrations['registrations'].fillna(0)

#Считаем конверсию
visits_and_registrations['conversion'] = round(visits_and_registrations['registrations']/visits_and_registrations['visits']*100, 0)

#Сохраняем в формате json
visits_and_registrations.to_json('conversion.json', orient = 'columns')

print(visits_and_registrations.head(10))


         date platform  visits  registrations  conversion
0  2023-03-01  android     264            218        83.0
1  2023-03-01      ios      78             67        86.0
2  2023-03-01      web     333             78        23.0
3  2023-03-02  android     106             91        86.0
4  2023-03-02      ios     103             76        74.0
5  2023-03-02      web     462             52        11.0
6  2023-03-03  android     251            218        87.0
7  2023-03-03      ios      57             54        95.0
8  2023-03-03      web    1221             66         5.0
9  2023-03-04  android     193            119        62.0


Задание 4

In [114]:
#Читаем данные из csv файла
ads_info = pd.read_csv('ads.csv')

#Добавляем столбец с датой
ads_info['date_ymd'] = pd.to_datetime(ads_info['date']).dt.date

#Группируем по дате и рекламной компании, находим стоимость рекламных кампаний по дням
ads_grouped = ads_info.groupby(['date_ymd', 'utm_campaign']).agg({'cost':'sum'}).reset_index()

#Соединяем таблицу конверсий и рекламы, переименовываем столбцы
ads = visits_and_registrations.merge(ads_grouped, left_on = 'date', right_on = 'date_ymd', how = 'left') 
ads = ads[['date', 'visits', 'registrations', 'cost', 'utm_campaign']]
ads.rename(columns = {'date':'date_group'}, inplace = True )
ads['cost'] = ads['cost'].fillna(0)
ads['utm_campaign'] = ads['utm_campaign'].fillna('none')

#Сохраняем в формате json
ads.to_json('ads.json', orient = 'columns')
print(ads.head(5))
# print(visits_and_registrations.head(200))
# print(ads_grouped.head(10))


   date_group  visits  registrations   cost              utm_campaign
0  2023-03-01     264            218  238.0  virtual_reality_workshop
1  2023-03-01      78             67  238.0  virtual_reality_workshop
2  2023-03-01     333             78  238.0  virtual_reality_workshop
3  2023-03-02     106             91  164.0  virtual_reality_workshop
4  2023-03-02     103             76  164.0  virtual_reality_workshop


Задание 5

In [110]:
total_visits = ads.groupby(['date_group']).agg({'visits':'sum'})
print(total_visits.head(5))

            visits
date_group        
2023-03-01     675
2023-03-02     671
2023-03-03    1529
2023-03-04    2083
2023-03-05    2321
